In [2]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [6]:
!pip install bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00


In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

要开始微调Jamba，我们需要加载将用于微调的数据集。在本示例中，我们将使用Abrate存储库的“english_quotes”数据集。以下是加载数据集的示例代码：

In [1]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


datasets库的load_dataset函数使我们可以轻松访问和加载所需的数据集。我们需要指定存储库和数据集名称，以及我们要用于训练的拆分。

接下来，我们需要配置分词器，并加载预训练的Jamba模型。分词器负责将文本数据转换为模型可以理解的格式。以下是配置分词器并加载模型的示例代码：

In [15]:
pip install -U bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int4_skip_modules=["mamba"]
)

tokenizer = AutoTokenizer.from_pretrained("ai21labs/Jamba-v0.1")

model = AutoModelForCausalLM.from_pretrained(
    "ai21labs/Jamba-v0.1",
    trust_remote_code=True,
    device_map='auto',
    attn_implementation="flash_attention_2",
    quantization_config=quantization_config,
    use_mamba_kernels=True
)

Unused kwargs: ['llm_int4_skip_modules']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


model.safetensors.index.json:   0%|          | 0.00/114k [00:00<?, ?B/s]

model-00001-of-00021.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00021.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00003-of-00021.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model-00008-of-00021.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00009-of-00021.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 4884.67 MB. The target location /root/.cache/huggingface/hub/models--ai21labs--Jamba-v0.1/blobs only has 3507.33 MB free disk space.
  warnings.warn(


model-00010-of-00021.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

在此代码片段中，我们使用transformers库的AutoTokenizer和AutoModelForCausalLM类加载Jamba分词器和模型。我们还使用BitsAndBytesConfig配置了量化设置，以启用4位量化，并指定在量化期间要跳过的模块。

为了控制微调过程，我们需要定义训练参数。这些参数指定了训练的各种超参数和设置。以下是如何定义训练参数的示例：

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="adamw_8bit",
    max_grad_norm=0.3,
    weight_decay=0.001,
    warmup_ratio=0.03,
    gradient_checkpointing=True,
    logging_dir='./logs',
    logging_steps=1,
    max_steps=50,
    group_by_length=True,
    lr_scheduler_type="linear",
    learning_rate=2e-3
)

在此代码片段中，我们创建了TrainingArguments类的一个实例，并指定了各种参数，如输出目录、训练时期的数量、批量大小、优化器、学习率等。根据您的具体需求和可用资源调整这些参数。

LoRA（低秩适应）是一种用于高效微调大型语言模型（如Jamba）的技术。它通过仅更新模型的一小部分参数来实现参数效率的微调。以下是如何为微调Jamba配置LoRA的示例：

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    init_lora_weights=False,
    r=8,
    target_modules=["embed_tokens", "x_proj", "in_proj", "out_proj"],
    task_type="CAUSAL_LM",
    bias="none"
)

在此代码片段中，我们使用peft库的LoraConfig类创建了一个实例。我们指定了LoRA超参数，如lora_alpha、lora_dropout以及要应用LoRA的目标模块。根据您的具体需求和实验调整这些设置。

有了数据集、分词器、模型、训练参数和LoRA配置，我们现在可以创建训练器对象。训练器负责管理微调过程。以下是如何创建训练器的示例：

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    train_dataset=dataset,
    max_seq_length=256,
    dataset_text_field="quote",
)

在此代码片段中，我们使用trl库的SFTTrainer类创建了一个实例。我们将已加载的模型、分词器、训练参数、LoRA配置和训练数据集传递给训练器。我们还指定了最大序列长度和要从数据集中使用的文本字段。

一切准备就绪，我们现在可以开始微调过程了。只需在训练器对象上调用train方法即可：

In [ ]:
trainer.train()

这将启动微调过程，Jamba将开始从提供的数据集中进行学习。训练的进展将在控制台中显示，包括损失和其他相关指标。

完成微调过程后，您可以评估微调模型在验证数据集上的性能，或者使用它生成文本。要生成文本，您可以使用模型的generate方法，传入所需的提示和生成参数。

In [ ]:
generated_text = model.generate(
    prompt="从前有一个时间",
    max_length=100,
    num_return_sequences=1,
    temperature=0.7
)

根据您的特定需求和实验调整生成参数。